# Informe de Análisis de Densidad Espectral de Potencia (PSD) de la Pletismografía (PPG)

---

## 1. Introducción

La Densidad Espectral de Potencia (PSD) describe cómo se distribuye la **potencia de una señal** a través de sus frecuencias. En señales fisiológicas como la PPG, permite:

- Identificar componentes de la señal asociados con el pulso y su variabilidad.
- Detectar ruido y artefactos (movimiento, interferencia ambiental).
- Comparar diferentes métodos de estimación de PSD para determinar cuál es más estable y confiable según el tipo de señal y objetivo.

**Métodos analizados en este proyecto:**

- **Periodograma**
- **Welch**
- **Blackman-Tukey (BT)**

Todos se aplicaron a un segmento de PPG de aproximadamente **15.000 muestras (~2 minutos)**, para equilibrar resolución y tiempo de cálculo.

---

## 2. Experimentos realizados

### 2.1 Señal PPG en el tiempo

**Qué se hace:**  
Se grafica la señal original de PPG en función del tiempo (`resultados/Pletismografia_original_tiempo.png`).

**Base matemática simple:**

$$
x[n] \quad \text{vs} \quad t = \frac{n}{f_s}
$$

**Objetivo:**

- Tener un referente temporal de la señal.
- Detectar visualmente artefactos o irregularidades.

**Justificación:**  
Antes de hacer análisis espectral, es fundamental observar la señal para asegurar que **no hay picos inesperados, saturación o errores de adquisición**.

---

### 2.2 FFT de la señal PPG

**Qué se hace:**  
Se calcula la FFT de la señal completa y se grafica su magnitud (`resultados/FFT_Pletismografia.png`).

**Base matemática:**

$$
X[k] = \sum_{n=0}^{N-1} x[n] e^{-j 2 \pi k n / N}
$$

Se toma la magnitud:

$$
|X[k]|
$$

**Objetivo:**

- Observar la composición espectral directa de la PPG.
- Identificar frecuencias dominantes (ej.: ritmo cardíaco, ruido).

**Justificación:**  
La FFT permite ver todas las frecuencias presentes en la señal y sirve como **análisis preliminar antes de aplicar PSD**.

---

### 2.3 Periodograma

**Qué se hace:**  
Se aplica el periodograma con ventana Hamming (`resultados/Densidad_Espectral_de_Potencia_periodograma.png` y `resultados/AnchoBanda_Densidad_Espectral_de_Potencia_periodograma.png`).

**Base matemática:**

$$
PSD(f) = \frac{1}{N \cdot f_s} \big| FFT[x[n] \cdot w[n]] \big|^2
$$

donde \( w[n] \) es la ventana Hamming que suaviza los bordes y evita fugas espectrales.

**Objetivo:**

- Obtener la distribución de potencia de la señal de manera directa y sencilla.
- Visualizar y calcular el **ancho de banda**, es decir, el rango de frecuencias significativas.

**Justificación:**  
El periodograma es rápido y fácil de implementar, pero sensible al ruido, especialmente en señales fisiológicas cortas.

**Ventajas:**

- Método simple y directo.
- Buen primer vistazo de la PSD.

**Desventajas:**

- Alta varianza en estimaciones.
- Puede mostrar picos falsos si la señal es corta o ruidosa.

---

### 2.4 Método de Welch

**Qué se hace:**

- La señal se divide en **segmentos solapados** (2048 muestras, solapamiento 50%).
- Se calcula el periodograma de cada segmento y se promedian (`resultados/Densidad_Espectral_de_Potencia_welch.png` y `resultados/AnchoBanda_Densidad_Espectral_de_Potencia_welch.png`).

**Base matemática:**

$$
PSD_{Welch}(f) = \frac{1}{K} \sum_{k=1}^{K} PSD_{\text{segmento } k}(f)
$$

Promediando se reduce la **varianza**, obteniendo una PSD más suave y estable.

**Objetivo:**

- Estimar la PSD con menos ruido que el periodograma.
- Calcular ancho de banda de manera más confiable.

**Justificación:**  
Promediar varios segmentos reduce los picos aleatorios, ideal para **señales fisiológicas donde la estabilidad espectral es importante**.

**Ventajas:**

- PSD más suave y estable.
- Reduce efectos de ruido.
- Balance entre resolución y estabilidad.

**Desventajas:**

- Menor resolución en frecuencia si los segmentos son cortos.
- Requiere más cálculos que el periodograma simple.

---

### 2.5 Método Blackman-Tukey

**Qué se hace:**

- Se calcula la **autocorrelación limitada** de la señal.
- Se aplica ventana (Hamming) sobre la autocorrelación.
- Se calcula la FFT de la autocorrelación para obtener PSD (`resultados/Densidad_Espectral_de_Potencia_blackman_tukey.png` y `resultados/AnchoBanda_Densidad_Espectral_de_Potencia_blackman_tukey.png`).

**Base matemática:**

Autocorrelación:

$$
r_{xx}[m] = \sum_n x[n] x[n-m]
$$

PSD a partir de autocorrelación:

$$
PSD(f) = FFT[r_{xx}[m]]
$$

**Notas adicionales:**  

- El gráfico de Blackman-Tukey puede verse espejado por la simetría de la autocorrelación. Se analiza solo la **mitad positiva**.  
- La implementación en Python se basó en código MATLAB del libro de Monson H. Hayes, manteniendo la lógica de autocorrelación y ventana.

**Objetivo:**

- Obtener una PSD suavizada y reducir fluctuaciones.
- Ideal para señales donde interesa la forma general del espectro, más que los picos exactos.

**Ventajas:**

- Suavidad y estabilidad.
- Menos sensible a picos aleatorios.

**Desventajas:**

- Requiere más cálculos que los métodos anteriores.
- Menor resolución si el número de retardos \( M \) es pequeño.
- Gráfico espejado si no se selecciona la mitad positiva del espectro.

---

## 3. Análisis de Ancho de Banda

**Qué se hace:**  
Se calcula el rango de frecuencias donde la PSD supera el **5% del máximo** (`resultados/ancho_de_banda_Densidad_Espectral_de_Potencia.csv`).  
Esto indica las **frecuencias relevantes de la señal**.

**Objetivo:**

- Comparar métodos de PSD de forma cuantitativa.
- Determinar qué métodos ofrecen estimación más consistente del ancho de banda.

**Justificación:**  

- Ritmo cardíaco típico: 0.5–4 Hz  
- Ruido: >50 Hz  

Esto permite separar **señal de pulso** de ruido y evaluar precisión de cada método.

---

## 4. Conclusiones generales

| Método           | Resolución | Suavidad | Sensibilidad al ruido | Complejidad |
|-----------------|------------|----------|---------------------|-------------|
| Periodograma     | Alta       | Baja     | Alta                | Baja        |
| Welch            | Media      | Media    | Media               | Media       |
| Blackman-Tukey   | Media-Baja | Alta     | Baja                | Alta        |

**Síntesis:**

- **Periodograma:** rápido y simple, útil para primer vistazo, sensible al ruido.  
- **Welch:** suavizado y estable, reduce picos falsos, balance resolución/robustez.  
- **Blackman-Tukey:** muy suave y estable, útil para señales cortas o ruidosas, más costoso computacionalmente.

---

## 5. Conclusiones finales (resumen conceptual y guía de uso)

**Resumen fácil de recordar:**

- **Periodograma:**  
  - Exploración rápida.  
  - Útil para ver rápidamente frecuencias dominantes.  
  - Riesgo de picos falsos en señales cortas o ruidosas.

- **Welch:**  
  - PSD suave y estable.  
  - Reduce ruido y picos aleatorios.  
  - Ideal para señales fisiológicas largas donde se necesita robustez.

- **Blackman-Tukey:**  
  - PSD muy suave, gracias a autocorrelación.  
  - Mejor para señales cortas o con mucho ruido.  
  - Útil cuando interesa **la forma general del espectro** más que los picos exactos.

**Tabla rápida de uso:**

| Método           | Cuándo usarlo                                         |
|-----------------|------------------------------------------------------|
| Periodograma     | Exploración rápida y primer vistazo de la PSD       |
| Welch            | Señales largas o ruidosas, cuando se busca estabilidad |
| Blackman-Tukey   | Señales cortas o muy ruidosas, para suavidad máxima |

**Regla práctica:**  

- Para rapidez → **Periodograma**  
- Para robustez y suavidad → **Welch**  
- Para señales difíciles (cortas/ruidosas) → **Blackman-Tukey**

**Tip conceptual:**  

- **Periodograma:** “rápido pero ruidoso”  
- **Welch:** “equilibrio entre suavidad y detalle”  
- **Blackman-Tukey:** “suave y seguro, aunque más lento”

---

*Este informe está listo para ser usado como notebook de referencia para análisis espectral de señales fisiológicas, combinando teoría, fórmulas y recomendaciones prácticas.*
